In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from cosy.models import CosyNetMultiInput
from cosy.losses import trace_norm_loss, MaskedMeanSquaredError

In [2]:
df = pd.read_csv("/Users/tompope/Documents/python_enviroment/SoftwareDevelopment/soft_parameter_sharing/data/desulferization_data_w_tobacc.csv")
df.head()

,Unnamed: 0,framework,CO2_tp_lod,N2_tp_lod,SO2_tp_lod,RDF_electronegativity_2.00,RDF_electronegativity_2.00.1,RDF_electronegativity_2.01,RDF_electronegativity_2.03,RDF_electronegativity_2.04,...,RDF_vdWaalsVolume_28.53,RDF_vdWaalsVolume_29.01,RDF_vdWaalsVolume_29.51,RDF_vdWaalsVolume_30.00,PLD,LCD,density,VF,SAv,SAg
0,0,GIQXUA_clean,1.918899,0.017520,2.747258,0.000605,0.000606,0.000607,0.000609,0.000614,...,0.0,0.0,0.0,0.0,3.86870,4.89334,1.792830,0.13844,2675.750,1492.470
1,1,JALPAQ_clean,0.050226,0.027788,0.008087,0.000861,0.000870,0.000888,0.000916,0.000954,...,0.0,0.0,0.0,0.0,2.90553,3.97550,1.660880,0.01992,841.869,506.881
2,2,SAQQEH_clean,2.006302,0.249367,0.533491,0.000771,0.000781,0.000802,0.000833,0.000876,...,0.0,0.0,0.0,0.0,9.13279,10.56525,0.914389,0.35942,2030.210,2220.290
3,3,BEFNEH_clean,1.036156,0.033717,1.343768,0.000899,0.000899,0.000900,0.000903,0.000910,...,0.0,0.0,0.0,0.0,3.33494,4.26940,2.165640,0.07290,1864.990,861.172
4,4,GELVID_clean,0.552087,0.064768,0.027879,0.000820,0.000831,0.000852,0.000884,0.000928,...,0.0,0.0,0.0,0.0,2.43534,3.61901,1.574130,0.02776,1334.150,847.547


In [3]:
'''
  - read data into dataframe
  - generate class data
  - extract and split feature and class data
  - scale feature data
'''


df = df.sample(frac=1).reset_index(drop=True)
df.drop(columns=['Unnamed: 0'], inplace=True)
# df = df[df.framework.str.startswith('str_')]


train_df_full, test_df = train_test_split(df, test_size=0.2, random_state=2020)

train_df, val_df = train_test_split(train_df_full, test_size=0.1, random_state=2020)

X_train = train_df.iloc[:, 4:].to_numpy().astype(np.float32)
X_train_CO2 = train_df.iloc[:, 4:-6].to_numpy().astype(np.float32)

y_CO2 = train_df.CO2_tp_lod.to_numpy().astype(np.float32)
y_N2 = train_df.N2_tp_lod.to_numpy().astype(np.float32)
y_SO2 = train_df.SO2_tp_lod.to_numpy().astype(np.float32)

X_val = val_df.iloc[:, 4:].to_numpy().astype(np.float32)
X_val_CO2 = val_df.iloc[:, 4:-6].to_numpy().astype(np.float32)
y_CO2_val = val_df.CO2_tp_lod.to_numpy().astype(np.float32)
y_N2_val = val_df.N2_tp_lod.to_numpy().astype(np.float32)
y_SO2_val = val_df.SO2_tp_lod.to_numpy().astype(np.float32)

X_test = test_df.iloc[:, 4:].to_numpy().astype(np.float32)
X_test_CO2 = test_df.iloc[:, 4:-6].to_numpy().astype(np.float32)
y_CO2_test =  test_df.CO2_tp_lod.to_numpy().astype(np.float32)
y_N2_test = test_df.N2_tp_lod.to_numpy().astype(np.float32)
y_SO2_test = test_df.SO2_tp_lod.to_numpy().astype(np.float32)


# y_CO2[100:1000] = -1

sc = StandardScaler()


X_train = sc.fit_transform(X_train)
X_val = sc.transform(X_val)
X_test = sc.transform(X_test)

In [4]:
df.head()

,framework,CO2_tp_lod,N2_tp_lod,SO2_tp_lod,RDF_electronegativity_2.00,RDF_electronegativity_2.00.1,RDF_electronegativity_2.01,RDF_electronegativity_2.03,RDF_electronegativity_2.04,RDF_electronegativity_2.07,...,RDF_vdWaalsVolume_28.53,RDF_vdWaalsVolume_29.01,RDF_vdWaalsVolume_29.51,RDF_vdWaalsVolume_30.00,PLD,LCD,density,VF,SAv,SAg
0,VOZVIR01_clean,0.243592,0.118635,0.031885,0.000714,0.000723,0.000740,0.000766,0.000802,0.000848,...,0.000000,0.000000,0.000000,0.000000,5.57973,8.87661,1.346080,0.24614,2074.24,1540.95
1,str_m2_o25_o28_f0_nbo_sym_111_faps_mpnn_charges,0.183181,0.317703,0.048971,0.000758,0.000768,0.000790,0.000824,0.000871,0.000933,...,0.001182,0.001237,0.001512,0.001240,16.49478,25.98730,0.177762,0.81886,1013.90,5703.65
2,CUNWEO_clean,0.952602,0.138539,0.699509,0.000740,0.000753,0.000777,0.000816,0.000870,0.000940,...,0.000000,0.000000,0.000000,0.000000,6.48545,7.07961,1.231460,0.18212,1539.74,1250.34
3,str_m2_o13_o22_f0_pcu_sym_64_faps_mpnn_charges,0.434624,0.228501,0.054699,0.000782,0.000793,0.000816,0.000852,0.000900,0.000963,...,0.000000,0.000000,0.000000,0.000000,6.08678,8.27915,0.822739,0.28594,2461.09,2991.33
4,str_m3_o3_o9_f0_nbo_sym_117_faps_mpnn_charges,0.336151,0.251086,0.079782,0.000745,0.000756,0.000778,0.000811,0.000857,0.000916,...,0.000605,0.000155,0.000286,0.000012,12.95626,16.16932,0.289265,0.68368,1568.94,5423.89


In [5]:
input_ = tf.keras.Input(shape=(X_train.shape[1]))
x = tf.keras.layers.Dense(1200, activation='relu')(input_)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(400, activation='relu')(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(500, activation='relu')(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(1, activation='relu')(x)

model = tf.keras.Model(inputs=input_, outputs=x)

Metal device set to: Apple M1


2023-02-20 14:44:34.111893: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-20 14:44:34.112338: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
input_ = tf.keras.Input(shape=(X_train_CO2.shape[1]))
x = tf.keras.layers.Dense(1200, activation='relu')(input_)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(400, activation='relu')(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(500, activation='relu')(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(1, activation='relu')(x)

modelco2 = tf.keras.Model(inputs=input_, outputs=x)

In [7]:
cosy_model = CosyNetMultiInput(
    model_config=[model.get_config(), modelco2.get_config(), model.get_config()],
    number_models=3,
    scalar=1.0,
    max_layer_cutoff=-1,
    min_layer_cutoff=1
)

In [8]:
opt = tf.keras.optimizers.Adam(learning_rate=0.00025)

losses = {
    'output_1':MaskedMeanSquaredError(),
    'output_2': MaskedMeanSquaredError(),
    'output_3': MaskedMeanSquaredError(),
    }

R2 = tfa.metrics.RSquare()

cosy_model.compile(
            optimizer=opt,
            loss=losses,
            metrics=[R2],
            )

In [9]:
cosy_model.fit(
    x=[X_train, X_train_CO2, X_train], y=[y_N2, y_CO2, y_SO2],
    validation_data=([X_val, X_val_CO2, X_val], [y_N2_val, y_CO2_val, y_SO2_val]),
    batch_size=32, epochs = 4, verbose=1
)

Epoch 1/4


2023-02-20 14:44:34.558499: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-20 14:44:35.732045: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


878/878 [==============================] - ETA: 0s - loss: 7.6086 - output_1_loss: 0.0216 - output_2_loss: 0.3518 - output_3_loss: 0.6907 - output_1_r_square: -1.8708 - output_2_r_square: 0.0515 - output_3_r_square: 0.4757 - scaled_soft_loss: 2.1797

2023-02-20 14:44:57.414230: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


878/878 [==============================] - 24s 25ms/step - loss: 7.6086 - output_1_loss: 0.0216 - output_2_loss: 0.3518 - output_3_loss: 0.6907 - output_1_r_square: -1.8708 - output_2_r_square: 0.0515 - output_3_r_square: 0.4757 - scaled_soft_loss: 2.1797 - val_loss: 0.9545 - val_output_1_loss: 0.0157 - val_output_2_loss: 0.2996 - val_output_3_loss: 0.5499 - val_output_1_r_square: -1.1535 - val_output_2_r_square: 0.1323 - val_output_3_r_square: 0.5718 - val_scaled_soft_loss: 0.0298
Epoch 2/4
353/878 [===========>..................] - ETA: 12s - loss: 0.9910 - output_1_loss: 0.0173 - output_2_loss: 0.3047 - output_3_loss: 0.5632 - output_1_r_square: -1.2400 - output_2_r_square: 0.1454 - output_3_r_square: 0.5648 - scaled_soft_loss: 0.0353

KeyboardInterrupt: 

In [ ]:
models = cosy_model.get_models()

In [ ]:
from sklearn.metrics import r2_score
cosy_model.evaluate([X_test, X_test_CO2, X_test], [y_N2_test, y_CO2_test, y_SO2_test])
predictions = cosy_model.predict([X_test, X_test_CO2, X_test])

R2 = tfa.metrics.RSquare()

pred1 = np.array(predictions[0]).reshape(-1)
pred2 = np.array(predictions[1]).reshape(-1)
pred3 = np.array(predictions[2]).reshape(-1)


print(r2_score(y_N2_test, pred1))
print(r2_score(y_CO2_test, pred2))
print(r2_score(y_SO2_test, pred3))

In [ ]:
y = y_N2

In [ ]:
np.shape(y)

In [ ]:
y_N2

In [ ]:
masker = tf.not_equal(y, -1)

In [ ]:
masker

In [ ]:
y = tf.boolean_mask(y, masker)

In [ ]:
y

In [ ]:
models = cosy_model.get_models()

In [ ]:
models[2].layers[1].weights

In [ ]:
for idx, params in enumerate(zip(*[layer.weights for layer in cosy_model.task_nets.layers])):
    if "kernel" in params[1].name and idx > 0:
        print(params)

In [10]:
cosy_model

In [17]:
all_weights = [
            layer
            for layer in cosy_model.get_models()[0].layers
            # if "kernel" in layer.weights
        ]

TypeError: Cannot convert 'kernel' to EagerTensor of dtype float

In [25]:
for layer in cosy_model.get_models()[0].layers:
    # if "kernel" in layer.weights:
    print(layer.weights)

[]
[<tf.Variable 'dense/kernel:0' shape=(345, 1200) dtype=float32, numpy=
array([[-0.01514168,  0.04906128,  0.01946328, ..., -0.02703662,
         0.05521812, -0.0523131 ],
       [-0.0532867 , -0.04593742,  0.0477429 , ...,  0.01824075,
         0.05604015,  0.04978141],
       [ 0.00157958,  0.04079344, -0.02539153, ...,  0.05700975,
        -0.04277142, -0.03173221],
       ...,
       [ 0.03064074, -0.01263676, -0.01946   , ..., -0.042841  ,
        -0.00439206,  0.03778353],
       [-0.01934196, -0.01862761,  0.01766437, ..., -0.04586348,
         0.02547225,  0.02775244],
       [ 0.02913373, -0.03622467, -0.01667222, ...,  0.03764926,
         0.02948315, -0.06398466]], dtype=float32)>, <tf.Variable 'dense/bias:0' shape=(1200,) dtype=float32, numpy=
array([ 0.01152364, -0.00109134,  0.02565357, ..., -0.00709428,
       -0.0101028 , -0.01753984], dtype=float32)>]
[]
[<tf.Variable 'dense_1/kernel:0' shape=(1200, 400) dtype=float32, numpy=
array([[-0.03462267,  0.05343335,  0.0301

In [16]:
cosy_model.get_models()[0].layers